1. Add headers = ["Document Source","Requirement Number","CTBC Requirement Head","CTBC Description","TCS BaNCS Component","Module Name","TCS understanding (General Requirement Description)","Fitement Analysis (Standard/Partial Gap/Gap)","Customisattion Efforts Estimate (XL/L/M/S)","Solution Summary","Remark, If Any"]
2. write word content to excel
    - save all the requirement number
    - write original content (split requirement number and description and add 7 empty string elements at last)
    - add requirement head
3. excel 第一個 sheet 改 style (all cells with borders, headers with #002060 as background and #FFFFFF as font color and bold font, content cells with #E7E6E6)
4. 加 table 到 excel sheets (add borders)


In [14]:
import docx
import openpyxl
from openpyxl.styles import Side, Border, Alignment
import pandas as pd
import numpy as np
import re
import string

In [15]:
file_name = 'Navi-F2-030-UR-1.14-Gold inventory position management'
doc = docx.Document(file_name + '.docx')
all_par = doc.paragraphs
len(all_par)

116

In [16]:
doc_source = re.match("^[A-Za-z0-9]*-[A-Za-z0-9]*-[A-Za-z0-9]*-[A-Za-z0-9]*-[A-Za-z0-9_\.]*", file_name)[0]
dict = {} # every numbering content dictionary for adding requirement head use
num_list = [] # document numbering
listx = [] # excel dataframe
listx.append(["Document Source","Requirement Number","CTBC Requirement Head","CTBC Description","TCS BaNCS Component","Module Name","TCS understanding (General Requirement Description)","Fitement Analysis (Standard/Partial Gap/Gap)","Customisattion Efforts Estimate (XL/L/M/S)","Solution Summary","Remark, If Any"])
for par in all_par:
    text = par.text.strip()
    if (text != ''):
        number = re.match("^[1-9-\.]+[\.]?", text)
        content = re.split("^([1-9-\.]+[\.]?)", text)
        curr = []
        if number:
            num_list.append(number[0])
            curr = [content[1], "", content[2].strip()]
            dict[number[0]] = content[2].strip()
        elif len(num_list) != 0: # eliminate title
            curr = [num_list[len(num_list)-1], "", content[0].strip()]
        if len(curr) != 0:
            # find all parents and add header
            header = ''
            num = curr[0]
            meet_digit = False
            for i in range(len(num)-1, -1, -1): 
                if num[i].isdigit():
                    meet_digit = True
                elif meet_digit: # meet the next . or -
                    meet_digit = False
                    parent = num[:i+1]
                    parent = parent[:-1] if (parent[:-1] in dict) else parent
                    # if num[i] == '-':
                    #     parent = parent[:-1]+'.'
                    if parent in dict: 
                        header = (dict[parent]+'-' if len(header) !=0 else dict[parent])+header
            curr[1] = header
            # add to dataframe
            listx.append([doc_source] + curr + ["","","","","","","",""])
df = pd.DataFrame(listx)
css_headers = 'background-color: #002060; color: #FFFFFF; font-weight: bold; border-color: black; border-width: 1px; border-style: solid;'
css_content = 'background-color: #E7E6E6; color: black; border-color: black; border-width: 1px; border-style: solid;'
(df.style.apply(lambda col: np.where(col.index == 0, css_headers, css_content)) # alternating rows
).to_excel(file_name+'.xlsx', engine='openpyxl', index=False, header=False, sheet_name=doc_source)

In [17]:
workbook = openpyxl.load_workbook(filename=file_name+'.xlsx')
if doc_source in workbook.sheetnames[0]:
        sheet = workbook[workbook.sheetnames[0]]
        sheet.column_dimensions['A'].width = 25
        sheet.column_dimensions['C'].width = 45
        sheet.column_dimensions['D'].width = 55
        for row in sheet.iter_rows():
                for cell in row:
                        cell.alignment = Alignment(wrap_text=True)

count = 0
css_borders = 'border-color: black; border-width: 1px; border-style: solid;'
side = Side(border_style='thin', color='000000')
alphabet = list(string.ascii_uppercase)
for table in doc.tables:
        new_sheet = workbook.create_sheet(str(count))
        # for tr,row in zip(table.rows, new_sheet.iter_rows()):
        #         for tc,cell in zip(tr.cells, row):
        #                 if tc.text:
        #                         cell.value = tc.text
        #                         cell.border = Border(top=side, bottom=side, left=side, right=side)
        for i,row in enumerate(table.rows):
                for j,cell in enumerate(row.cells):
                        if i == 0:
                                new_sheet.column_dimensions[alphabet[j]].width = 25
                        new_sheet.cell(i+1,j+1).value = cell.text
                        new_sheet.cell(i+1,j+1).border = Border(top=side, bottom=side, left=side, right=side)
                        new_sheet.cell(i+1,j+1).alignment = Alignment(wrap_text=True)
        count+=1
workbook.save(filename=file_name+'.xlsx')


In [18]:
### helper
# header = ''
# meet_digit = False
# num = '1-3.2.1'
# for i in range(len(num)-1, -1, -1): # find all parents
#     if num[i].isdigit():
#         meet_digit = True
#     elif meet_digit: # meet the next . or -
#         meet_digit = False
#         parent = num[:i+1]
#         print(parent)
#         parent = parent[:-1] if (parent[:-1] in dict) else parent
#         if parent in dict: 
#             header = (dict[parent]+'-' if len(header) !=0 else dict[parent])+header
#             print(header)

